
### Търсене и извличане на информация. Приложение на дълбоко машинно обучение
### Стоян Михов
### Зимен семестър 2025/2026

## Упражнение 13

За да работи програмата трябва корпуса от публицистични текстове за Югоизточна Европа,
да се намира разархивиран в наддиректорията, в която е програмата (виж упражнение 2).

Преди да се стартира програмата е необходимо да се активира съответното обкръжение с командата:
conda activate tii

In [1]:
import sys
import nltk
from nltk.corpus import PlaintextCorpusReader
import numpy as np
import torch
import random
import math

In [2]:
class progressBar:
    def __init__(self ,barWidth = 50):
        self.barWidth = barWidth
        self.period = None
    def start(self, count):
        self.item=0
        self.period = int(count / self.barWidth)
        sys.stdout.write("["+(" " * self.barWidth)+"]")
        sys.stdout.flush()
    def tick(self):
        if self.item>0 and self.item % self.period == 0:
            progress = self.item // self.period
            sys.stdout.write(f"\r[{'-'* progress}{' '*(self.barWidth - progress)}]")
            sys.stdout.flush()
        self.item += 1
    def stop(self):
        sys.stdout.write("\r["+("-" * self.barWidth)+"]\n")

###  Разбиване на корпус на тестов и тренинг

In [3]:
def extractDictionary(corpus, limit=20000):
	pb = progressBar()
	pb.start(len(corpus))
	dictionary = {}
	for doc in corpus:
		pb.tick()
		for w in doc:
			if w not in dictionary: dictionary[w] = 0
		dictionary[w] += 1
	L = sorted([(w,dictionary[w]) for w in dictionary], key = lambda x: x[1] , reverse=True)
	if limit > len(L): limit = len(L)
	words = [ w for w,_ in L[:limit] ] + [unkToken] + [padToken]
	word2ind = { w:i for i,w in enumerate(words)}
	pb.stop()
	return words, word2ind

def splitSentCorpus(fullSentCorpus, testFraction = 0.1):
	random.seed(42)
	random.shuffle(fullSentCorpus)
	testCount = int(len(fullSentCorpus) * testFraction)
	testSentCorpus = fullSentCorpus[:testCount]
	trainSentCorpus = fullSentCorpus[testCount:]
	return testSentCorpus, trainSentCorpus

### Зареждане на корпуса

In [4]:
corpus_root = '../JOURNALISM.BG/C-MassMedia'
myCorpus = PlaintextCorpusReader(corpus_root, '.*\.txt')
startToken = '<s>'
endToken = '</s>'
unkToken = '<unk>'
padToken = '<pad>'

corpus = [ [startToken] + [w.lower() for w in sent] + [endToken] for sent in myCorpus.sents()]
words, word2ind = extractDictionary(corpus)

testCorpus, trainCorpus  = splitSentCorpus(corpus, testFraction = 0.01)

batchSize = 32
d_model = 128
n_head = 4

#device = torch.device("cpu")
device = torch.device("cuda:0")
# device = torch.device("mps")

[--------------------------------------------------]


### Позиционно влагане


\begin{aligned}
\mathrm{PE}(i, 2k) &= \sin\!\left(\frac{i}{10000^{\,2k / d}}\right) \\
\mathrm{PE}(i, 2k+1) &= \cos\!\left(\frac{i}{10000^{\,2k / d}}\right)
\end{aligned}


In [5]:
class PositionalEncoding(torch.nn.Module):

	def __init__(self, d_model, max_len = 5000):
		super().__init__()

		pe = torch.zeros(1, max_len, d_model)

		position = torch.arange(max_len).unsqueeze(0).unsqueeze(2)
		div_term = ( 10000.0 ** (torch.arange(0, d_model, 2)/d_model) ).unsqueeze(0).unsqueeze(0)
		pe[0,:,0::2] = torch.sin(position / div_term)
		pe[0,:,1::2] = torch.cos(position / div_term)
		self.register_buffer('pe', pe)

	def forward(self, x):
		x = x + self.pe[:,:x.shape[1],:]		# x.shape = (batch_size, seq_len, embedding_dim)
		return x

### Attention




$$\mathrm{Attention}(Q, K, V) = \mathrm{softmax}\!\left(\frac{Q K^{\top}}{\sqrt{d_k}}\right) V $$


In [6]:
class Attention(torch.nn.Module):
	def __init__(self, d_model, d_keys, d_values):
		super(Attention, self).__init__()

		self.d_model, self.d_keys, self.d_values = d_model, d_keys, d_values
		self.scale = 1 / (d_keys ** 0.5)

		self.Wq_net = torch.nn.Linear(d_model, d_keys)
		self.Wk_net = torch.nn.Linear(d_model, d_keys)
		self.Wv_net = torch.nn.Linear(d_model, d_values)
		self.Wo_net = torch.nn.Linear(d_values, d_model)

	def forward(self, input, attn_mask=None):

		batch_size = input.shape[0]
		seq_len = input.shape[1]
		# input.shape = (batch_size, seq_len, d_model)

		q = self.Wq_net(input)
		k = self.Wk_net(input)
		v = self.Wv_net(input)
		# q.shape = (batch_size, seq_len, d_keys)
		# k.shape = (batch_size, seq_len, d_keys)
		# v.shape = (batch_size, seq_len, d_values)

		k = k.transpose(1, 2)
		# k.shape = (batch_size, d_keys, seq_len)

		attn_score = torch.matmul(q, k)
		# attn_score.shape = (batch_size, seq_len, seq_len)

		attn_score = attn_score * self.scale

		if attn_mask is not None:
			# attn_mask.shape = (seq_len, seq_len)
			attn_score = attn_score.masked_fill(
				attn_mask.unsqueeze(0), -float('inf')
			)

		attn_prob = torch.nn.functional.softmax(attn_score, dim=2)
		# attn_prob.shape = (batch_size, seq_len, seq_len)

		attn_vec = torch.matmul(attn_prob, v)
		# attn_vec.shape = (batch_size, seq_len, d_values)

		attn_out = self.Wo_net(attn_vec)
		# attn_out.shape = (batch_size, seq_len, d_model)

		return attn_out


### Multi Head Attention



$$
\mathrm{MultiHead}(Q, K, V)
= \mathrm{Concat}(\text{head}_1, \dots, \text{head}_h)\, W^O
$$

\begin{aligned}
where
\end{aligned}

$$
\text{head}_i
= \mathrm{Attention}\!\left(
Q W_i^Q,\;
K W_i^K,\;
V W_i^V
\right)
$$


In [7]:
class MultiHeadAttn(torch.nn.Module):
	def __init__(self, n_head, d_model, d_keys, d_values):
		super(MultiHeadAttn, self).__init__()
	
		self.n_head, self.d_model, self.d_keys, self.d_values = n_head, d_model, d_keys, d_values
		self.scale = 1 / (d_keys ** 0.5)
	
		self.Wq_net = torch.nn.Linear(d_model, n_head * d_keys)
		self.Wk_net = torch.nn.Linear(d_model, n_head * d_keys)
		self.Wv_net = torch.nn.Linear(d_model, n_head * d_values)
		self.Wo_net = torch.nn.Linear(n_head * d_values, d_model)
	
	def forward(self, input, attn_mask = None):

		n_head, d_model, d_keys, d_values = self.n_head, self.d_model, self.d_keys, self.d_values

		batch_size = input.shape[0]
		seq_len = input.shape[1]
		# input.shape = (batch_size, seq_len, d_model)
		head_q = self.Wq_net(input)
		head_k = self.Wk_net(input)
		head_v = self.Wv_net(input)
		# {head_q,head_k}.shape = (batch_size, seq_len, n_head * d_keys)
		# head_v.shape = (batch_size, seq_len, n_head * d_values)
		
		q = head_q.view(batch_size, seq_len, n_head, d_keys).transpose(1,2)
		k = head_k.view(batch_size, seq_len, n_head, d_keys).permute(0,2,3,1)
		v = head_v.view(batch_size, seq_len, n_head, d_values).transpose(1,2)
		# q.shape = (batch_size, n_head, seq_len, d_keys)
		# k.shape = (batch_size, n_head, d_keys, seq_len)
		# v.shape = (batch_size, n_head, seq_len, d_values)
		
		attn_score = torch.matmul(q, k)
		# attn_score.shape = (batch_size, n_head, seq_len, seq_len)
			
		attn_score = attn_score * self.scale

		if attn_mask is not None:
			# attn_mask = (seq_len, seq_len)
			attn_score = attn_score.masked_fill(attn_mask.unsqueeze(0).unsqueeze(1), -float('inf'))

		attn_prob = torch.nn.functional.softmax(attn_score, dim=3)
		# attn_prob.shape = (batch_size, n_head, seq_len, seq_len)
		attn_vec = torch.matmul(attn_prob, v)
		# attn_vec.shape = (batch_size, n_head, seq_len, d_values)

		attn_vec = attn_vec.transpose(1,2).flatten(2,3)
		# attn_vec.shape = (batch_size, seq_len, n_head * d_values)

		# linear projection
		attn_out = self.Wo_net(attn_vec)
		# attn_out = (batch_size, seq_len, d_model)
		return attn_out

### Transformer Cell

In [8]:
class Transformer_cell(torch.nn.Module):

	def __init__(self, n_head, d_model, d_keys, d_values, d_ff, dropout):

		super().__init__()
		self.MHA = MultiHeadAttn(n_head, d_model, d_keys, d_values)
		self.layer_norm_1 = torch.nn.LayerNorm(d_model)
		self.dropout_1 = torch.nn.Dropout(dropout)
		self.W1 = torch.nn.Linear(d_model,d_ff)
		self.W2 = torch.nn.Linear(d_ff,d_model)
		self.layer_norm_2 = torch.nn.LayerNorm(d_model)
		self.dropout_2 = torch.nn.Dropout(dropout)
	
	def forward(self, x, mask = None):

		# x.shape = (batch_size, seq_len, d_model)
		# mask.shape = (seq_len, seq_len)
		z1 = self.MHA(x, mask)
		z2 = self.layer_norm_1(x + self.dropout_1(z1))

		z3 = self.W2(torch.nn.functional.relu(self.W1(z2))) ## Feed Forward
		y = self.layer_norm_2(z2 + self.dropout_2(z3))

		return y

In [15]:
class TransformerLanguageModel(torch.nn.Module):
	def __init__(self, n_head, d_model, word2ind, unkToken, padToken, maxlen=5000):
		super(TransformerLanguageModel, self).__init__()
		self.word2ind = word2ind
		self.unkTokenIdx = word2ind[unkToken]
		self.padTokenIdx = word2ind[padToken]
		
		self.embed = torch.nn.Embedding(len(word2ind), d_model)
		
		self.pos_embed = PositionalEncoding(d_model)
		self.dropout1 = torch.nn.Dropout(0.1)
		
		self.Transformer = Transformer_cell(n_head, d_model, d_model//n_head, d_model//n_head, d_model*n_head, 0.1)

		pos = torch.arange(maxlen)
		mask = pos.unsqueeze(0)>pos.unsqueeze(1)
		self.register_buffer('mask', mask)

		self.projection = torch.nn.Linear(d_model,len(word2ind))
	
	def preparePaddedBatch(self, source):
		device = next(self.parameters()).device
		m = max(len(s) for s in source)
		sents = [[self.word2ind.get(w,self.unkTokenIdx) for w in s] for s in source]
		sents_padded = [ s+(m-len(s))*[self.padTokenIdx] for s in sents]
		return torch.tensor(sents_padded, dtype=torch.long, device=device)	# shape=(batch_size, seq_len)

	def forward(self, source):
		X = self.preparePaddedBatch(source) # X.shape=(batch_size, seq_len)
		seq_len = X.shape[1]
		E = self.embed(X[:,:-1])	# E.shape = (batch_size, seq_len-1, d_model)
		input = self.dropout1(self.pos_embed(E))
		
		batch_mask = self.mask[:seq_len-1,:seq_len-1]
		tr = self.Transformer(input, batch_mask)

		Z = self.projection(tr.flatten(0,1))
		Y_bar = X[:,1:].flatten(0,1)
		H = torch.nn.functional.cross_entropy(Z,Y_bar,ignore_index=self.padTokenIdx)
		return H
	
	@torch.no_grad()
	def generate(self, source, eosToken, max_len=100):
		self.eval()
		device = next(self.parameters()).device

		eos_idx = self.word2ind[eosToken]

		# convert words to indices
		seq = [self.word2ind.get(w, self.unkTokenIdx) for w in source]

		for _ in range(max_len):
			X = torch.tensor([seq], dtype=torch.long, device=device)
			seq_len = X.shape[1]

			E = self.embed(X)
			input = self.dropout1(self.pos_embed(E))

			tr = input
			batch_mask = self.mask[:seq_len, :seq_len]

			tr = self.Transformer(input, batch_mask)

			logits = self.projection(tr[:, -1])   # last token
			# remove unknown and padding tokens from the vocabulary
			logits[0, self.unkTokenIdx] = -float('inf')
			logits[0, self.padTokenIdx] = -float('inf')
			# Sample
			next_token = torch.multinomial(torch.softmax(logits, dim=-1), 1).item()
	
			if next_token == eos_idx:
				break

			seq.append(next_token)

		# convert back to words
		ind2word = {v: k for k, v in self.word2ind.items()}
		return [ind2word[i] for i in seq]



In [16]:
lm = TransformerLanguageModel(n_head, d_model, word2ind, unkToken, padToken).to(device)
optimizer = torch.optim.Adam(lm.parameters(), lr=0.001)

idx = np.arange(len(trainCorpus), dtype='int32')
np.random.shuffle(idx)

In [17]:
for b in range(0, len(idx), batchSize):
	batch = [ trainCorpus[i] for i in idx[b:min(b+batchSize, len(idx))] ]
	H = lm(batch)
	optimizer.zero_grad()
	H.backward()
	optimizer.step()
	if b % 10 == 0:
		print(b, '/', len(idx), H.item())

0 / 388284 10.049528121948242
160 / 388284 9.160099029541016
320 / 388284 8.16933536529541
480 / 388284 7.490640163421631
640 / 388284 6.990583419799805
800 / 388284 6.420965194702148
960 / 388284 6.114800930023193
1120 / 388284 6.028353214263916
1280 / 388284 5.965117454528809
1440 / 388284 6.326777935028076
1600 / 388284 5.912780284881592
1760 / 388284 5.896554946899414
1920 / 388284 6.140764236450195
2080 / 388284 5.671100616455078
2240 / 388284 5.83384370803833
2400 / 388284 5.770162582397461
2560 / 388284 5.726028919219971
2720 / 388284 5.8228631019592285
2880 / 388284 5.72743034362793
3040 / 388284 5.4687089920043945
3200 / 388284 5.522647857666016
3360 / 388284 5.364433765411377
3520 / 388284 5.507852554321289
3680 / 388284 5.360968112945557
3840 / 388284 5.458451271057129
4000 / 388284 5.5192766189575195
4160 / 388284 5.466897964477539
4320 / 388284 5.518957614898682
4480 / 388284 5.4018635749816895
4640 / 388284 5.306214809417725
4800 / 388284 5.247514247894287
4960 / 388284 5

In [18]:
def perplexity(lm, testCorpus, batchSize):
	H = 0.
	c = 0
	for b in range(0,len(testCorpus),batchSize):
		batch = testCorpus[b:min(b+batchSize, len(testCorpus))]
		l = sum(len(s)-1 for s in batch)
		c += l
		with torch.no_grad():
			H += l * lm(batch)
	return math.exp(H/c)

In [19]:
perplexity(lm, testCorpus, batchSize)

38.48666997283384

In [20]:
print(lm.generate([startToken], endToken, max_len=20))

['<s>', 'белград', ',', 'сърбия', '--', 'сръбският', 'президент', 'борис', 'тадич', 'и', 'грузия', 'откри', 'полицията', 'в', 'понеделник', '(', '17', 'февруари', '),', 'за', 'да']


In [21]:
print(lm.generate([startToken, "днес", "българия"], endToken, max_len=20))

['<s>', 'днес', 'българия', 'ще', 'съживява', 'достъпа', 'до', 'голяма', 'сърбия', 'и', 'черна', 'гора', 'през', 'кампанията', 'и', 'не', 'го', 'обвинява', 'политическото', 'си', 'обръщение', 'към', 'него']
